In [ ]:
from astropy.io import fits
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import pyarrow.parquet as parquet
import fast_histogram as fh
import time
import dask.dataframe as dd
from scipy import stats


In [ ]:
def dosemap_ranges(radius):
    slop = 400 - radius
    return [slop, 3200 - slop], [slop, 3200 - slop]

In [ ]:
# load data for being on detector / in dosemap frame, convert to dosemap coords

nf = parquet.read_table("/home/bekah/gPhoton2/test_data/e23456/e23456-nd-b00.parquet",
                       columns=['col', 'row', 'ra', 'dec', 'response']).to_pandas()

#nf = parquet.read_table("/home/bekah/gPhoton2/test_data/e23456/e00781-nd-b00.parquet",
#                        columns=['col', 'row', 'ra', 'dec', 'response']).to_pandas()

In [ ]:
bstart = time.time()


In [ ]:
### FILTERING ARRAY ### 
start_time = time.time()

print("filtering data")

# this is an antiquated cutoff from when radius was 600 for 2dhist
nf = nf[(nf['row']*4 >= -200) & (nf['row']*4 <= 3400) & (nf['col']*4 >= -200) & (nf['col']*4 <= 3400)]

nf['row_rnd'] = nf['row'].round().astype(int)
nf['col_rnd'] = nf['col'].round().astype(int)

nf = nf[(nf['row_rnd'] <= 4799) & (nf['col_rnd'] <= 4799) & (nf['ra']!= 0) & (nf['dec']!= 0)]
nf = nf[(nf['ra'] <= 4799) & (nf['dec'] <= 4799)]

nf = nf[(nf['col_rnd']<800)&(nf['row_rnd']<800)]


mask = pd.notna(nf['ra'])
nf = nf[mask]

end_time = time.time()

duration = end_time - start_time
print(f"it took {duration:.4f} seconds")

In [ ]:
### COUNT HIST ### 
start_time = time.time()

print("finished filtering data, making histogram now")
# make 2d histogram and filter for dark and hotpsots. "hotspots" will inevitably contain bright stars

hist = fh.histogram2d(
       nf['col'], nf['row'],bins=800, range=([0,800],[0,800])).astype('float32')

print("done with histogram")

end_time = time.time()

duration = end_time - start_time
print(f"it took {duration:.4f} seconds")

In [ ]:
# scipy stats binned statistics 

start_time = time.time()

ret_dec = stats.binned_statistic_2d(nf['col'], nf['row'], [nf['dec'], nf['ra']], 'std', bins=800)

std_dec = ret_dec.statistic[0]
std_ra = ret_dec.statistic[1]

combo_std = std_ra + std_dec 

end_time = time.time()

duration = end_time - start_time
print(f"it took {duration:.4f} seconds")

In [ ]:
### CALCULATING DISPERSION & RESPONSE ### 
start_time = time.time()

print("calculating dispersion & mean response")

dispersion = nf.groupby(['col_rnd', 'row_rnd']).agg({
    'ra': 'std',
    'dec': 'std',
    'response': 'mean'
}).reset_index()

counts = nf.groupby(['col_rnd', 'row_rnd']).size().reset_index(name='count')

dispersion = pd.merge(dispersion, counts, on=['col_rnd', 'row_rnd'])

ra_dec_disp = np.zeros((800, 800))

ra_dec_disp[dispersion['col_rnd'], dispersion['row_rnd']] = dispersion['ra']+dispersion['dec']

end_time = time.time()

duration = end_time - start_time
print(f"it took {duration:.4f} seconds")

In [ ]:
### DISP ARRAY ### 

start_time = time.time()

# making np array of dispersion ra / dec 
print("making dispersion array")

dispersion = dispersion[(dispersion['col_rnd']<800)&(dispersion['row_rnd']<800)]
max_col = dispersion['col_rnd'].max()
max_row = dispersion['row_rnd'].max()

ra_dec_disp = np.zeros((800, 800))

ra_dec_disp[dispersion['col_rnd'], dispersion['row_rnd']] = abs(dispersion['ra'])+abs(dispersion['dec'])

end_time = time.time()

duration = end_time - start_time
print(f"it took {duration:.4f} seconds")

In [ ]:
### RESPONSE ARRAY ### 

start_time = time.time()

# making np array of response
print("making response array")

dispersion = dispersion[(dispersion['col_rnd']<800)&(dispersion['row_rnd']<800)]
#max_col = dispersion['col_rnd'].max()
#max_row = dispersion['row_rnd'].max()

response_map = np.zeros((800, 800))

response_map[dispersion['col_rnd'], dispersion['row_rnd']] = dispersion['response']

end_time = time.time()

duration = end_time - start_time
print(f"it took {duration:.4f} seconds")

In [ ]:
### MASKING ### 

start_time = time.time()
print("masking")

# density mask
density_mask = hist > 400 # not doing low density bc we have response, otherwise gets mixed with background

# dispersion mask
disp_mask = ra_dec_disp > .014

# response map 
response_mask = response_map < .7 

# combo mask
result_array = np.zeros(hist.shape, dtype=int)+1

result_array[density_mask & disp_mask] = 0
result_array[response_mask] = 0

# saving mask to fits 
hdu = fits.PrimaryHDU(result_array)
hdul = fits.HDUList([hdu])
hdul.writeto('mask.fits', overwrite=True)

end_time = time.time()

duration = end_time - start_time
print(f"it took {duration:.4f} seconds")

In [ ]:
bend = time.time()

print(f"total time: {bstart-bend}")

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(ra_dec_disp, interpolation=None,origin='lower')
plt.colorbar()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

plt.scatter(dispersion['ra']+dispersion['dec'],dispersion['count'],c='grey',s=0.5)
plt.xlabel('ra / dec dispersion')
plt.ylabel('count')
